In [1]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

import matplotlib.pyplot as plt

2024-06-11 18:23:39.025506: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-11 18:23:42.856641: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [12]:
# import os
# from ultralytics import YOLO
# import cv2

# def filter_images_with_objects(yolo_model, input_folder, output_folder, targetclasses):
#     # Load the YOLO model
#     model = YOLO(yolo_model)

#     # Create the output folder if it doesn't exist
#     if not os.path.exists(output_folder):
#         os.makedirs(output_folder)

#     for filename in os.listdir(input_folder):
#         if filename.endswith(".jpg") or filename.endswith(".png"):
#             image_path = os.path.join(input_folder, filename)
#             image = cv2.imread(image_path)

#             # Run the YOLO model on the image
#             results = model(image)

#             # Check if the desired object is detected
#             for result in results:
#                 found = False
#                 for bbox in result.boxes.data:
#                     # bbox format: [x_min, y_min, x_max, y_max, confidence, class_id]
#                     class_id = int(bbox[5])
#                     if model.names[class_id] in targetclasses:
#                         found = True
#                         # Draw bounding box
#                         x_min, y_min, x_max, y_max = map(int, bbox[:4])
#                         cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
#                         cv2.putText(image, model.names[class_id], (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

#                 if found:
#                     # Save the image with annotations
#                     output_path = os.path.join(output_folder, filename)
#                     print(f"Saving annotated image to: {output_path}")
#                     cv2.imwrite(output_path, image)
#                     break

# # Example usage
# yolo_model = 'yolov8x.pt'  # You can specify your model file path here
# input_folder = 'Data Collection'
# output_folder = 'Filtered Data Collection'
# targetclasses = ['Military Vehicle', 'truck', 'car', 'boat', 'SUV', 'tank', 'Armored Vehicle', 'helicopter', 'aircraft', 'person', 'bus']

# filter_images_with_objects(yolo_model, input_folder, output_folder, targetclasses)

In [14]:
import os
from ultralytics import YOLO
import cv2
import uuid

def filter_and_annotate_images(yolo_model, input_folder, output_folder, target_classes):
    # Load the YOLO model
    model = YOLO(yolo_model)

    # Create the output folders if they don't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    annotations_folder = os.path.join(output_folder, 'annotations')
    if not os.path.exists(annotations_folder):
        os.makedirs(annotations_folder)

    for filename in os.listdir(input_folder):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_path = os.path.join(input_folder, filename)
            image = cv2.imread(image_path)
            height, width = image.shape[:2]

            # Run the YOLO model on the image
            results = model(image)

            for result in results:
                found = False
                annotation_content = []
                for bbox in result.boxes.data:
                    # bbox format: [x_min, y_min, x_max, y_max, confidence, class_id]
                    class_id = int(bbox[5])
                    if model.names[class_id] in target_classes:
                        found = True
                        x_min, y_min, x_max, y_max = map(int, bbox[:4])
                        
                        # Calculate YOLO format values
                        x_center = (x_min + x_max) / 2.0 / width
                        y_center = (y_min + y_max) / 2.0 / height
                        box_width = (x_max - x_min) / width
                        box_height = (y_max - y_min) / height
                        
                        annotation_content.append(f"{class_id} {x_center} {y_center} {box_width} {box_height}")
                
                if found:
                    # Generate a unique filename
                    unique_filename = str(uuid.uuid4())
                    image_output_path = os.path.join(output_folder, f"{unique_filename}.jpg")
                    annotation_output_path = os.path.join(annotations_folder, f"{unique_filename}.txt")

                    # Save the image
                    cv2.imwrite(image_output_path, image)
                    # Save the annotation
                    with open(annotation_output_path, 'w') as f:
                        f.write("\n".join(annotation_content))
                    print(f"Saved {image_output_path} and its annotations.")

# Example usage
yolo_model = 'yolov8x.pt'  # You can specify your model file path here
input_folder = 'extracted_frames_from_v7'
output_folder = 'Filtered Data Collection'
target_classes = ['car','bus','truck','boat','bus','airplane']

filter_and_annotate_images(yolo_model, input_folder, output_folder, target_classes)


0: 384x640 (no detections), 15976.5ms
Speed: 16.6ms preprocess, 15976.5ms inference, 3.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 15542.2ms
Speed: 11.0ms preprocess, 15542.2ms inference, 3.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 15174.1ms
Speed: 10.8ms preprocess, 15174.1ms inference, 3.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 15344.6ms
Speed: 12.3ms preprocess, 15344.6ms inference, 3.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 15241.9ms
Speed: 12.1ms preprocess, 15241.9ms inference, 3.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 18200.2ms
Speed: 13.8ms preprocess, 18200.2ms inference, 2.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 train, 14683.8ms
Speed: 10.3ms preprocess, 14683.8ms inference, 6.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 truck, 14832.0